<a href="https://colab.research.google.com/github/himsgpt/GenAI_usecase/blob/main/mcp_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MCP SERVER Options
You’ve already used:

1. Gradio with MCP extension ✅
Simple and great for exposing Python functions with GUI and MCP support: demo.launch(mcp_server=True)
2. fastmcp.server
A minimal and direct MCP server using FastAPI or HTTP. You can use this if you're building a backend server without UI:from fastmcp.server import Tool, Server
3. Custom server (FastAPI, Flask, etc.)
Using the mcp protocol classes directly, you can implement a fully custom server.


MCP CLIENT Options
1. fastmcp.Client ✅
The standard async-compatible MCP client.
Uses:
.list_tools()
.call(tool_name, input_dict)
Requires async/await

2. smolagents.MCPClient
Wraps fastmcp.Client, made for agents and tool orchestration (like AutoGPT-style apps).

3. LangChain, AutoGen, OpenAgents (advanced)
You can use MCP with agent frameworks like:
LangChain's tool calling
OpenAgents
AutoGen from Microsoft
These are more complex and typically async, and often integrate smolmodels, openai, llama.cpp, etc.

In [ ]:
! pip install "gradio[mcp]" textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00


In [ ]:
## Gradio server

In [ ]:
import json
import gradio as gr
from textblob import TextBlob

def sentiment_analysis(text: str) -> str:
    """
    Analyze the sentiment of the given text.

    Args:
        text (str): The text to analyze

    Returns:
        str: A JSON string containing polarity, subjectivity, and assessment
    """
    blob = TextBlob(text)
    sentiment = blob.sentiment

    result = {
        "polarity": round(sentiment.polarity, 2),  # -1 (negative) to 1 (positive)
        "subjectivity": round(sentiment.subjectivity, 2),  # 0 (objective) to 1 (subjective)
        "assessment": "positive" if sentiment.polarity > 0 else "negative" if sentiment.polarity < 0 else "neutral"
    }

    return json.dumps(result)

# Create the Gradio interface
demo = gr.Interface(
    fn=sentiment_analysis,
    inputs=gr.Textbox(placeholder="Enter text to analyze..."),
    outputs=gr.Textbox(),  # Changed from gr.JSON() to gr.Textbox()
    title="Text Sentiment Analysis",
    description="Analyze the sentiment of text using TextBlob"
)

# Launch the interface and MCP server
if __name__ == "__main__":
    demo.launch(mcp_server=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d86967b42dc81b244c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)

🔨 MCP server (using SSE) running at: https://d86967b42dc81b244c.gradio.live/gradio_api/mcp/sse


## Option 1

In [ ]:
!pip install fastmcp llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4067776 sha256=419baf5c8b1ee20f9d50a7f7d518c40957b42ce5303a7235bf9640dbe306c864
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python
  Attempting un

In [31]:
from fastmcp.client import Client
import asyncio

MCP_SERVER_URL = "https://d86967b42dc81b244c.gradio.live/gradio_api/mcp/sse"
client = Client(MCP_SERVER_URL)
# tools = asyncio.run(client.list_tools())
# print("Available tools:", tools)

In [19]:
async def list_and_call():
    async with client:
        tools = await client.list_tools()
        print("Available tools:", tools)

        # Call the sentiment analysis tool
        text_input = "I love how easy this is to use!"
        result = await client.call_tool("predict", {"text": text_input})
        print("Sentiment Result:", result)

# Run the async function in Colab
await list_and_call()

Available tools: [Tool(name='predict', description='Analyze the sentiment of the given text.', inputSchema={'type': 'object', 'properties': {'text': {'type': 'string', 'description': 'The text to analyze'}}}, annotations=None)]
Sentiment Result: [TextContent(type='text', text='{"polarity": 0.52, "subjectivity": 0.72, "assessment": "positive"}', annotations=None)]


## Option 2

In [21]:
# install npx
!pip install "smolagents[mcp]" mcp fastmcp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 13.4 MB/s eta 0:00:00


In [22]:
import gradio as gr
import os

from mcp import StdioServerParameters
from smolagents import InferenceClientModel, CodeAgent, ToolCollection, MCPClient

In [34]:
mcp_client = MCPClient(
    {"url": "https://d86967b42dc81b244c.gradio.live/gradio_api/mcp/sse",
     "transport": "sse"} # This is the MCP server we created in the previous section
)
tools = mcp_client.get_tools()
# print("Available tools:", tools)
for tool in tools:
    print(f"Name: {tool.name}")
    print(f"Description: {tool.description}")
    # print(f"Input Schema: {tool.input_schema}")
    print("-" * 40)

Name: predict
Description: Analyze the sentiment of the given text.
----------------------------------------


In [47]:
async def list_and_call():
    async with client:
        tools = await client.list_tools()
        print("✅ Available tools:")
        for tool in tools:
            print("-", tool.name)

        # Choose the tool by name and call it
        tool_name = "predict"  # Based on your Gradio app
        tool_args = {"text": "I am not happy today"}

        # Call the tool
        # result = await client.callTool(tool_name, tool_args)
        # print("✅ Sentiment Result:", result)

# ✅ Run the async function in Colab
await list_and_call()

✅ Available tools:
- predict


## LLM inclusion now

In [ ]:
##

In [ ]:
model = InferenceClientModel(token=os.getenv("HF_TOKEN"))
agent = CodeAgent(tools=[*tools], model=model)

In [ ]:
demo = gr.ChatInterface(
    fn=lambda message, history: str(agent.run(message)),
    type="messages",
    examples=["Prime factorization of 68"],
    title="Agent with MCP Tools",
    description="This is a simple agent that uses MCP tools to answer questions."
)

demo.launch()